In this demo, we will show how to use:

1. `ActionSet` to specify how consumers can change the inputs of a machine learning model

2. `Flipset` to create actionable adverse action notices for consumers who are denied loans

3. `RecourseAuditor` to measure the feasibility and difficulty of recourse before deployment


# Preliminaries

You can install actionable recourse by typing:

In [ ]:
! pip install actionable-recourse

We start by building a simple logistic regression for loan approval for the demo. 

We use the **default of credit card clients** dataset from the [UCI ML repository](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients)). 


In [ ]:
import recourse as rs
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from IPython.core.display import display, HTML
pd.options.display.float_format = '{:,.3f}'.format

# import data
url = 'https://raw.githubusercontent.com/ustunb/actionable-recourse/master/examples/paper/data/credit_processed.csv'
df = pd.read_csv(url)
y, X = df.iloc[:, 0], df.iloc[:, 1:]

# train a classifier
clf = LogisticRegression(max_iter = 1000, C=1./1000, solver='liblinear', penalty='l1')
clf.fit(X, y)
yhat = clf.predict(X)

The outcome variable is defined as:

> y[i] = 1 if a person repays a credit card loan

> y[i] = 0 if a person defaults on a credit card loan

The model `clf` predicts this outcome using the following input variables and weights


In [ ]:
pd.Series(clf.coef_[0], index=X.columns).to_frame('Coefficients')

,Coefficients
Married,0.000
Single,0.000
Age_lt_25,0.000
Age_in_25_to_40,0.000
Age_in_40_to_59,0.000
Age_geq_60,0.000
EducationLevel,0.332
MaxBillAmountOverLast6Months,0.000
MaxPaymentAmountOverLast6Months,0.000
MonthsWithZeroBalanceOverLast6Months,0.000


# Modeling Actions with `ActionSet`

The first step for any recourse-related analysis is to how individuals can change the input variables to a model – i.e., the set of feasible **actions.** We created `ActionSet` to make this as easy as possible for practitioners. 

Specifying action for each variable can take time. `ActionSet` can speed it up, by infering information from a sample of points.

In [ ]:
A = rs.ActionSet(X) 
print(A)

+---------------------------------------+----------------+------------+------------+----------------+----------------+-----------+-----------+-----------+-----+---------+
|                                  name |  variable type | actionable | compatible | step direction | flip direction | grid size | step type | step size |  lb |      ub |
+---------------------------------------+----------------+------------+------------+----------------+----------------+-----------+-----------+-----------+-----+---------+
|                               Married | <class 'bool'> |       True |        nan |              0 |            nan |         2 |  absolute |       1.0 | 0.0 |     1.0 |
|                                Single | <class 'bool'> |       True |        nan |              0 |            nan |         2 |  absolute |       1.0 | 0.0 |     1.0 |
|                             Age_lt_25 | <class 'bool'> |       True |        nan |              0 |            nan |         2 |  absolute |   

Users can then customize variables using familiar API commands


In [ ]:
# Variables that can't be changed
A['Married'].actionable = False
A[['Age_lt_25', 'Age_in_25_to_40', 'Age_in_40_to_59', 'Age_geq_60']].actionable = False
A[['TotalMonthsOverdue', 'TotalOverdueCounts', 'HistoryOfOverduePayments']].actionable = False
print(A)

+---------------------------------------+----------------+------------+------------+----------------+----------------+-----------+-----------+-----------+-----+---------+
|                                  name |  variable type | actionable | compatible | step direction | flip direction | grid size | step type | step size |  lb |      ub |
+---------------------------------------+----------------+------------+------------+----------------+----------------+-----------+-----------+-----------+-----+---------+
|                               Married | <class 'bool'> |      False |        nan |              0 |            nan |         2 |  absolute |       1.0 | 0.0 |     1.0 |
|                                Single | <class 'bool'> |       True |        nan |              0 |            nan |         2 |  absolute |       1.0 | 0.0 |     1.0 |
|                             Age_lt_25 | <class 'bool'> |      False |        nan |              0 |            nan |         2 |  absolute |   

In [ ]:
# EducationLevel takes values from (0, 3), can be changed in increments of 1, and must increase
A['EducationLevel'].bounds = (0, 3)
A['EducationLevel'].step_size = 1
A['EducationLevel'].step_type = "absolute" 
A['EducationLevel'].step_direction = 1 

# TotalMonthsOverview takes values between 0 to 12, can only increase by 1 month, and must increase
A['TotalMonthsOverdue'].bounds = (0, 12)
A['TotalMonthsOverdue'].step_size = 1  
A['TotalMonthsOverdue'].step_type = "absolute"  

# 
#A['MonthsWithLowSpendingOverLast6Months'].bounds = (0, 4)


In [ ]:
A.df[['name', 'actionable', 'lb', 'ub']].assign(lb=lambda df: df['lb'].astype(int)).assign(ub=lambda df: df['ub'].astype(int)).style.hide_index()

name,actionable,lb,ub
Married,False,0,1
Single,True,0,1
Age_lt_25,False,0,1
Age_in_25_to_40,False,0,1
Age_in_40_to_59,False,0,1
Age_geq_60,False,0,1
EducationLevel,True,0,3
MaxBillAmountOverLast6Months,True,0,11321
MaxPaymentAmountOverLast6Months,True,0,5480
MonthsWithZeroBalanceOverLast6Months,True,0,4


# Generating Actionable Adverse Action Notices with `Flipset`

When our model is deployed, it will deny loans whenever `clf.predict(x) = 0`. This is the case for person i = 13

In [ ]:
x = X.values[[13]]
print('yhat[i] = %d' % clf.predict(x)[0])

yhat[i] = 0


In this case, we can search for actions in an `ActionSet` that would allow the  applicant to change their prediction. We collect actions that change different feature combinations with a `Flipset`. This runs *very* quickly

In [ ]:
fs = rs.Flipset(x, action_set = A, clf = clf)
fs.populate(enumeration_type = 'distinct_subsets', total_items = 5);

obtained 5 items in 0.1 seconds


We can then easily print these items into a table that can be shown to a consumer with the Flipset.to_html command:

In [ ]:
display(HTML(fs.to_html()))

Features to Change,Current Value,to,Required Value
MaxBillAmountOverLast6Months,2060,→,2166
MaxBillAmountOverLast6Months,2060,→,2166
MaxPaymentAmountOverLast6Months,100,→,110
MaxBillAmountOverLast6Months,2060,→,2166
MostRecentBillAmount,2010,→,1926
MaxBillAmountOverLast6Months,2060,→,2166
MostRecentPaymentAmount,100,→,105
MonthsWithLowSpendingOverLast6Months,0,→,1


`Flipset` is designed to search over all possible actions for a consumer. When `Flipset` does not find an action for a person, then this is definitive proof that the person does not have recourse.


In [ ]:
p_approval = .95
score_approval = np.log(p_approval / (1. - p_approval))
x = X.values[[649]]
fs = rs.Flipset(x, action_set = A, coefficients=clf.coef_[0], intercept=clf.intercept_[0] - score_threshold)
fs.populate(enumeration_type = 'distinct_subsets', total_items = 5);
display(HTML(fs.to_html()))

recovered all minimum-cost items
obtained 0 items in 0.0 seconds


# Recourse Verification with `RecourseAuditor`

Ideally, we want to ensure that we provide consumers with recourse ahead of time. One way to ensure this is to check if a model provides recourse to each consumer in the training dataset. To make this process simpler, we produced `RecourseAuditor`

In [ ]:
# Basic Recourse Verification with 1 Model
# Use the auditor on 100 points (live)
# It's super easy

# How many people are dnied
# How many have recourse?
# How difficult is that recourse?

from recourse import RecourseAuditor

ra = RecourseAuditor(
    action_set=A,
    coefficients=clf.coef_[0],
    intercept=clf.intercept_[0] - score_threshold,
    solver='python-mip'
)

audit_output = ra.audit(X.sample(100))

In [ ]:
audit_output['feasible'].value_counts()

True     97
False     2
Name: feasible, dtype: int64